目标检测之SPP-Net空间金字塔池化网络(Spatial Pyramid Pooling Net)
===

# 1.为何引入SPP
我们知道在现有的CNN中，对于结构已经确定的网络，需要输入一张固定大小的图片，比如$224 \times 224，32 \times 32,96 \times 96$等。这样对于我们希望检测各种大小的图片的时候，需要经过裁剪，或者缩放等一系列操作，这样往往会降低识别检测的精度。于是“空间金字塔池化”变出现了，这个算法的牛逼之处，在于使得我们构建的网络，可以输入任意大小的图片，不需要经过裁剪缩放等操作，只要你喜欢，任意大小的图片都可以。

CNN需要固定图片输入的大小，是因为全连接层的存在。因此空间金字塔池化，要解决的就是从卷积层到全连接层之间的一个过度。

# 2.算法描述
![images](Images/03/02/01_02_001.png)
这张图从小往上看，最下面是输入的图片input_image,经过CNN之后，在全连接层之前，放入SPP层。举例如下，比如在CNN之后，输出的特征图大小为$1,512,16,12$，分别为batch_size, channels, height, width.

## 2.1.特征图分层
如果将特征图分就分为1层，那么这一层的大小将来就是$1 \times 1$；如果分为2层，那么两层的大小分别为$1 \times 1, 2 \times 2$；如果分为3层，那么三层的大小分别为$1 \times 1,2 \times 2, 4 \times 4$，以此类推，如果分为n层，那么每层的大小跟层级的关系如下
$$S_i=2^{i-1}$$
相当于SPP的关键就是需要对不同大小的特征图，采取一定的办法，保证他们分层之后的大小是一样的，这样送入全连接层的的大小是一样的

## 2.2.池化
从大图变为小图，我们使用的是池化算法(最大池化或平均值化)。那么既然不同大小的特征图，最后变为大小一致的特征图，所用的池化算法必然用到的ksize和stride是不一样的。我们需要计算的重点就是在这里

In [ ]:
import torch.nn
import math
import torch.nn.functional as F

class SPPLayer(torch.nn.Module):

    # 定义Layer需要的额外参数（除Tensor以外的）
    def __init__(self, size_list, pool_type='max_pool'):
        super(SPPLayer, self).__init__()

        self.size_list = size_list
        self.pool_type = pool_type

    # forward()的参数只能是Tensor(>=0.4.0) Variable(< 0.4.0)
    def forward(self, x):
        # num:样本数量 c:通道数 h:高 w:宽
        # num: the number of samples
        # c: the number of channels
        # h: height
        # w: width
        num, c, h, w = x.size()
        print('original', x.size())
        #         print(x.size())
        for i, level in enumerate(self.size_list):

            '''
            The equation is explained on the following site:
            http://www.cnblogs.com/marsggbo/p/8572846.html#autoid-0-0-0
            '''
            kernel_size = (math.ceil(h / level), math.ceil(w / level))  # kernel_size = (h, w)
            padding = (
                math.floor((kernel_size[0] * level - h + 1) / 2), math.floor((kernel_size[1] * level - w + 1) / 2))

            zero_pad = torch.nn.ZeroPad2d((padding[1], padding[1], padding[0], padding[0]))
            x_new = zero_pad(x)

            h_new, w_new = x_new.size()[2:]

            kernel_size = (math.ceil(h_new / level), math.ceil(w_new / level))
            stride = (math.floor(h_new / level), math.floor(w_new / level))

            if self.pool_type == 'max_pool':
                tensor = F.max_pool2d(x_new, kernel_size=kernel_size, stride=stride)
                tensor = tensor.view(num, -1)
            elif self.pool_type == 'avg_pool':
                tensor = F.avg_pool2d(x_new, kernel_size=kernel_size, stride=stride)
                tensor = tensor.view(num, -1)

            # 展开、拼接
            if (i == 0):
                x_flatten = tensor.view(num, -1)
            else:
                x_flatten = torch.cat((x_flatten, tensor.view(num, -1)), 1)

        return x_flatten

size_list就是分层的数值，如果特征图大小为$1,512,16,12$,分为3层之后，结果为$(1,512,1,1),(1,512,2,2),(1,512,4,4)$，也就是说不管最初的图大小是多大，经过SPP之后，都会变为这样的特征图

# 3.完整的SPP网络
## 3.1.找到候选框
首先通过选择性搜索，对待检测的图片进行搜索出2000个候选窗口。这一步和R-CNN一样。

## 3.2.特征提取
这一步就是和R-CNN最大的区别了，同样是用卷积神经网络进行特征提取，但是SPP-Net用的是金字塔池化。这一步骤的具体操作如下：
- 把整张待检测的图片，输入CNN中，进行一次性特征提取，得到feature maps，然后在feature maps中找到各个候选框的区域
- 再对各个候选框采用金字塔空间池化，提取出固定长度的特征向量。而R-CNN输入的是每个候选框，然后在进入CNN，因为SPP-Net只需要一次对整张图片进行特征提取，速度很快。R-CNN就相当于遍历一个CNN两千次，而SPP-Net只需要遍历1次
- 最后一步也是和R-CNN一样，采用SVM算法进行特征向量分类识别

主要是第二步会有一些问题，那就是如何在feature maps中找到原始图片中候选框的对应区域？

因为候选框是通过一整张原图片进行检测得到的，而feature maps的大小和原始图片的大小是不同的，feature maps是经过原始图片卷积、下采样等一系列操作后得到的。那么我们要如何在feature maps中找到对应的区域呢

假设$(x',y')$表示特征图上的坐标点，坐标点(x,y)表示原输入图片上的点，那么它们之间有如下转换关系：
$$(x,y)=(S*x', S*y')$$
其中S的就是CNN中所有的strides的乘积